# Fitting: Computing an NLL

We will be using Tensorflow's new eager mode, the new JIT static graph, and a classic API static graph to solve a different problem: fitting unbinned datasets. Like before, let's set up the data and then try a solution with Numpy:

In [ ]:
!nvidia-smi

In [ ]:
import numpy as np
import math

np.random.seed(42)

dist = np.hstack([
    np.random.normal(loc=1, scale=2., size=500_000),
    np.random.normal(loc=1, scale=.5, size=500_000)
])

In [ ]:
import torch, math

In [ ]:
d_dist = torch.tensor(dist)

In [ ]:
def gaussian(x, μ, σ):
    return 1/math.sqrt(2*np.pi*σ**2) * torch.exp(-(x-μ)**2/(2*σ**2))

def add(x, f_0, mean, sigma, sigma2):
    return f_0 * gaussian(x, mean, sigma) + (1 - f_0) * gaussian(x, mean, sigma2)

#@torch.jit.script
def nll(dist, f_0, mean, sigma, sigma2):
    return -torch.sum(torch.log(add(dist, f_0, mean, sigma, sigma2)))

In [ ]:
%%timeit
vals = [torch.tensor(v) for v in np.random.rand(4)]
nll(d_dist, *vals)

In [ ]:
device = torch.device("cuda")

In [ ]:
dev_dist = d_dist.to(device)

In [ ]:
%%timeit
vals = [torch.tensor(v).to(device) for v in np.random.rand(4)]
nll(dev_dist, *vals)
torch.cuda.synchronize()